In [ ]:
from __future__ import print_function
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import csv

In [2]:
def isNotNull(value):
    return value is not None and len(value)>0

In [3]:
#read the Bing Liu Lexicon dir
dict_pos = []
dict_neg = []
file = open('lexicon_dict/negative-words.txt','r')
for line in file:
    text= line.strip().lower();
    if (isNotNull(text)):
        dict_neg.append(text)
file.close()

file = open('lexicon_dict/positive-words.txt','r')
for line in file:
    text = line.strip().lower();
    if (isNotNull(text)):
        dict_pos.append(text)
file.close()

In [ ]:
#create spark context and the streaming context
sc = SparkContext("local[2]")
ssc = StreamingContext(sc,10)
IP = "localhost"
port=5555

#Logging control
sc.setLogLevel("ERROR")

lines = ssc.socketTextStream(IP, port)

In [ ]:
def calculate_sentiment_scores(text):
    tokens = text.split(" ")
    #sentiment_score = []
    
    neg_score = 0
    pos_score = 0
    
    for word in dict_neg:
        if (word in tokens):
            neg_score = neg_score + 1
    
    for word in dict_pos:
        if (word in tokens):
            pos_score= pos_score + 1
    
    sentiment_score = pos_score - neg_score
    return str(sentiment_score)

In [ ]:
with open('Tweets_scores.csv', 'w',encoding = "utf-8") as csvFile:
     
    fieldnames = ['Tweets', 'Sentiment_Score']
    writer = csv.DictWriter(csvFile, fieldnames=fieldnames)
    writer.writeheader()

In [ ]:
def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        # Get spark sql singleton context from the current context
        arr = rdd.collect()
        for i in range(0,len(arr),2):
            a = arr[i]
            b = arr[i + 1]
            print(a)
            print(b)
        
            with open('Tweets_scores.csv', 'a',encoding = "utf-8") as csvFile:
                fieldnames = ['Tweets', 'Sentiment_Score']
                writer = csv.DictWriter(csvFile, fieldnames=fieldnames)
                writer.writerow({'Tweets': a, 'Sentiment_Score': b })
    except Exception as e:
        print("Error: %s",e)

In [ ]:
#tweet.pprint()
tweets = lines.flatMap(lambda x: (x,calculate_sentiment_scores(x)))
tweets.foreachRDD(process_rdd)

In [ ]:
#Starting the Spark Streaming
ssc.start()

In [ ]:
ssc.awaitTermination()